In [1]:
import pandas as pd
from pathlib import Path
import pickle

In [2]:
upstream = ['read-csv']
product = None

In [3]:
# Parameters
upstream = {
    "read-csv": {
        "Y_col": "C:\\Users\\victor\\Documents\\notebooks\\output\\read-csv-Y_col.pkl",
        "df": "C:\\Users\\victor\\Documents\\notebooks\\output\\read-csv-df.pkl",
        "X_cols": "C:\\Users\\victor\\Documents\\notebooks\\output\\read-csv-X_cols.pkl",
        "nb": "C:\\Users\\victor\\Documents\\notebooks\\output\\read-csv.ipynb",
    }
}
product = {
    "df": "C:\\Users\\victor\\Documents\\notebooks\\output\\cleaning-and-converting-into-desired-format-df.pkl",
    "column_renamer": "C:\\Users\\victor\\Documents\\notebooks\\output\\cleaning-and-converting-into-desired-format-column_renamer.pkl",
    "nb": "C:\\Users\\victor\\Documents\\notebooks\\output\\cleaning-and-converting-into-desired-format.ipynb",
}


In [4]:
X_cols = pickle.loads(Path(upstream['read-csv']['X_cols']).read_bytes())
Y_col = pickle.loads(Path(upstream['read-csv']['Y_col']).read_bytes())
df = pickle.loads(Path(upstream['read-csv']['df']).read_bytes())

## Cleaning and converting into desired format

In [5]:
# These names are required by the library
column_renamer = {"location": "unique_id",
                  "date": "ds",
                  "new_cases": "y"}

In [6]:
# Library requires that all series end at the same time. Not many countries are lost this way
df = df[df.groupby("location")["date"].transform(max) == df["date"].max()]
minmaxes = df.groupby("location")["date"].agg([min, max, "count"])
minmaxes["proportions"] = (((minmaxes["max"] - minmaxes["min"]).dt.days + 1) / minmaxes["count"])
minmaxes[minmaxes["proportions"] != 1]

,min,max,count,proportions
location,,,,
Estonia,2020-01-06,2021-12-31,698,1.040115
Greece,2020-01-06,2021-12-31,678,1.070796
Latvia,2020-01-06,2021-12-31,675,1.075556
Norway,2020-02-12,2021-12-31,676,1.019231


In [7]:
# Aggregating data to be at a weekly level since daily was very messy
grouper_aggregator = {col: "mean" for col in X_cols}
grouper_aggregator.update({Y_col: "sum"})
df = df.groupby(["location", 
                 pd.Grouper(key='date', freq='W-MON')]).agg(grouper_aggregator).reset_index()

In [8]:
Path(product['column_renamer']).parent.mkdir(exist_ok=True, parents=True)
Path(product['column_renamer']).write_bytes(pickle.dumps(column_renamer))

Path(product['df']).parent.mkdir(exist_ok=True, parents=True)
Path(product['df']).write_bytes(pickle.dumps(df))

1279996